# Answering Business Questions on chinook.db
Chinook is a sample database based on real data from iTunes Library but with fictional names for customers, employees as well as random generated sales information. It basically describes sales of albums and music tracks in a series of different tables as seen in the schema below.

![alt text](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg "Chinook Schema")

The **objective** of this project is to **practice SQL Queries by answering a few common business questions**. The questions will be highlighted right before each query, and the findings will be displayed after the query. The first and following few queries however are simply for data exploration and for me to gather some basic understanding of some of the tables.

### Database Exploring
---

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
FROM customer
LIMIT 5;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [4]:
%%sql
SELECT *
FROM track
ORDER BY name
LIMIT 5;

 * sqlite:///chinook.db
Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
3027,"""40""",239,1,1,U2,157962,5251767,0.99
2918,"""?""",231,3,19,None,2782333,528227089,1.99
3412,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525: I. Allegro",281,2,24,Wolfgang Amadeus Mozart,348971,5760129,0.99
109,#1 Zero,11,1,4,"Cornell, Commerford, Morello, Wilk",299102,9731988,0.99
3254,#9 Dream,255,2,9,None,278312,4506425,0.99


In [5]:
%%sql
SELECT *
FROM invoice_line
ORDER BY track_id
LIMIT 5;

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
162,23,1,0.99,1
855,107,1,0.99,1
1956,252,1,0.99,1
2986,385,1,0.99,1
3231,414,1,0.99,1


In [6]:
%%sql
SELECT * FROM invoice
ORDER BY invoice_id
LIMIT 5;

 * sqlite:///chinook.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


In [7]:
%%sql
SELECT DISTINCT(quantity)
FROM invoice_line;

 * sqlite:///chinook.db
Done.


quantity
1


### What are the most common genres sold in Brazil?
---

In [8]:
%%sql
WITH tracks_count AS
    (
    SELECT
        il.track_id,
        COUNT(*) tracks_sold
    FROM invoice_line il
    JOIN invoice i USING(invoice_id)       -- To have access to billing_country
    WHERE i.billing_country = "Brazil"
    GROUP BY track_id
    ),

genre_absolute AS
    (SELECT
        t.genre_id genre_id,
        g.name genre,
        SUM(tc.tracks_sold) genre_sold
    FROM track t
    JOIN invoice_line il USING(track_id)   -- To have access to invoice table
    JOIN invoice i USING(invoice_id)       -- To have access to billing_country
    JOIN genre g USING(genre_id)
    JOIN tracks_count tc USING(track_id)
    WHERE i.billing_country = "Brazil"
    GROUP BY genre_id, genre
    ORDER BY genre_sold DESC
    )

SELECT
    genre,
    genre_sold sold,
    ROUND(100 * CAST(genre_sold AS FLOAT) / (SELECT SUM(genre_sold)
                                             FROM genre_absolute), 2) AS sales_pct
FROM genre_absolute;

 * sqlite:///chinook.db
Done.


genre,sold,sales_pct
Rock,235,41.81
Alternative & Punk,118,21.0
Metal,101,17.97
Jazz,22,3.91
Electronica/Dance,22,3.91
R&B/Soul,16,2.85
Latin,15,2.67
Classical,13,2.31
Easy Listening,7,1.25
Pop,6,1.07


The most common genre sold in Brazil is **Rock** followed by **Alternative & Punk** and **Metal**. There are by far at the top, with the next genre barely composing 4% of the sales (**Jazz** and **Electronica/Dance** tied).

### How much sales revenue have been achieved by each Employee?
---

In [9]:
%%sql

WITH customer_total AS
    (
    SELECT
        c.customer_id,
        c.support_rep_id,
        SUM(i.total) total_per_customer
    FROM customer c
    LEFT JOIN invoice i USING(customer_id)
    GROUP BY 1
    )

SELECT
    e.first_name || ' ' || e.last_name employee_name,
    e.title,
    SUM(total_per_customer) total_per_employee
FROM employee e
LEFT JOIN customer_total ct ON ct.support_rep_id = e.employee_id
GROUP BY employee_id;

 * sqlite:///chinook.db
Done.


employee_name,title,total_per_employee
Andrew Adams,General Manager,None
Nancy Edwards,Sales Manager,None
Jane Peacock,Sales Support Agent,1731.51
Margaret Park,Sales Support Agent,1584.0
Steve Johnson,Sales Support Agent,1393.92
Michael Mitchell,IT Manager,None
Robert King,IT Staff,None
Laura Callahan,IT Staff,None


**Jane Peacock** is the top sales person in the team, with **1731.51\$ as revenue**. Its also worth noticing that the sales team consists of only 3 people, even though the company has 8 Employees in total, 5 of which are not performing sales and so the revenue is 'None'.

### Sales analysis across all countries
---

In [10]:
%%sql
WITH raw_customer_invoice AS
    (
    SELECT
        c.customer_id,
        c.country,
        i.invoice_id,
        i.total,
        il.track_id
    FROM customer c
    JOIN invoice i USING(customer_id)
    JOIN invoice_line il USING(invoice_id)
    ),

total_summed_up AS
    (
    SELECT
        country,
        invoice_id,
        COUNT(track_id) count_tracks,
        COUNT(DISTINCT(invoice_id)) count_invoices,
        SUM(invoice_id) total_invoices,
        SUM(total) sum_purchases
    FROM raw_customer_invoice
    GROUP BY country
    ORDER BY country
    )

SELECT
    r.country,
    COUNT(DISTINCT(r.customer_id)) total_customers,
    t.count_tracks tracks_purchased,
    ROUND(CAST(SUM(r.total) AS FLOAT), 2) total_sales,
    ROUND(t.sum_purchases / COUNT(DISTINCT(r.customer_id)), 2) avg_purchase_customer,
    ROUND(t.sum_purchases / t.count_invoices, 2) avg_invoice_value
FROM raw_customer_invoice r
JOIN total_summed_up t USING(country)
GROUP BY country
ORDER BY country;

 * sqlite:///chinook.db
Done.


country,total_customers,tracks_purchased,total_sales,avg_purchase_customer,avg_invoice_value
Argentina,1,40,396.0,396.0,79.2
Australia,1,82,940.5,940.5,94.05
Austria,1,70,649.44,649.44,72.16
Belgium,1,61,567.27,567.27,81.04
Brazil,5,432,4059.0,811.8,66.54
Canada,8,541,5489.55,686.19,72.23
Chile,1,98,912.78,912.78,70.21
Czech Republic,2,276,3183.84,1591.92,106.13
Denmark,1,38,196.02,196.02,19.6
Finland,1,80,685.08,685.08,62.28


There are a lot of insights that can be taken from this, so we will list it down as topics:
- **Ireland** has the *highest average invoice value*
- **Czech Republic** has the *highest average purchase per customer*
- **USA** has the *highest total sales, tracks purchased and total customers*
- **Denmark** has the *lowest average purchase per customer, average invoice value and total sales*
- **Denmark and Argentina** have a similar number of tracks purchased but *Argentina has almost double in total sales*


### How many purchases include an entire album? And how many purchases select just a few tracks?
---

In [11]:
%%sql
WITH

-- Creating a specific table to gather album_id together with track_id and invoice_id
invoice_content AS
    (
    SELECT
        il.invoice_id,
        t.album_id,
        il.track_id
    FROM invoice_line il
    LEFT JOIN track t USING(track_id)
    ORDER BY invoice_id, album_id, track_id
    ),

-- Grouping invoice_id and album_id to make easier to count on the next step
albums_in_invoice AS
    (
    SELECT
        invoice_id,
        album_id
    FROM invoice_content
    GROUP BY invoice_id, album_id
    ORDER BY invoice_id, album_id
    ),

-- Generating a table counting how many albums contained in each invoice
-- (not necessarily the whole album is in there, just how many albums are present)
count_albums_per_invoice AS
    (
    SELECT
        invoice_id,
        COUNT(album_id) album_count
    FROM albums_in_invoice
    GROUP BY invoice_id
    ),

-- Now filtering and gathering only the invoices with maximum 1 album in it
-- This will be used in the last step
invoices_single_album AS 
    (   
    SELECT
        invoice_id
    FROM count_albums_per_invoice
    WHERE album_count = 1
    ),

-- Grouping albums and all it's tracks together
tracks_album AS
    (
    SELECT
        album_id,
        track_id
    FROM track
    ORDER BY album_id, track_id
    ),

-- Generating a code per album with a simple calculation of averaging the track_id
-- Each track_id is unique so generating an average of it per album results in a random
-- unique number per album, and we will use this 'code' to define which invoices have the
-- full album purchased (not just a few tracks)
track_album_code AS
    (
    SELECT
        album_id,
        AVG(track_id) code
    FROM tracks_album
    GROUP BY album_id
    ORDER BY album_id
    ),

-- Now utilizing the code generated, and the invoices_single_album
-- we can select only the invoices that have one full album purchased
purchases_full_album AS
    (    
    SELECT
        invoice_id,
        album_id,
        AVG(track_id) code
    FROM invoice_content
    WHERE invoice_id in invoices_single_album
    GROUP BY invoice_id, album_id
    HAVING code IN (SELECT code FROM track_album_code)
    ORDER BY invoice_id, album_id
    )
    
SELECT
    COUNT(cai.invoice_id) - COUNT(pf.invoice_id) single_tracks,
    COUNT(pf.invoice_id) albums
FROM count_albums_per_invoice cai
LEFT JOIN purchases_full_album pf USING(invoice_id);

 * sqlite:///chinook.db
Done.


single_tracks,albums
498,116


There are **498 invoices that purchased single tracks** in contrast to **116 invoices that purchased one full album**.

### Which artist is used in the most playlists?
---

In [12]:
%%sql
SELECT
    a.name artist_name,
    COUNT(pt.playlist_id) playlists
FROM artist a
JOIN album al USING(artist_id)
JOIN track t USING(album_id)
JOIN playlist_track pt USING(track_id)
GROUP BY artist_name
ORDER BY playlists DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


artist_name,playlists
Iron Maiden,516
U2,333
Metallica,296
Led Zeppelin,252
Deep Purple,226
Lost,184
Pearl Jam,177
Faith No More,145
Eric Clapton,145
Lenny Kravitz,143


**Iron Maiden** and **U2** figure at the top of most playlist choices with *more than 300 appearences*.

### How many tracks have been purchased vs not purchased?
---

In [13]:
%%sql
SELECT
    COUNT(DISTINCT(il.track_id)) purchased_tracks,
    COUNT(DISTINCT(t.track_id)) - COUNT(DISTINCT(il.track_id)) not_purchased_tracks,
    COUNT(DISTINCT(t.track_id)) total_tracks
FROM track t
LEFT JOIN invoice_line il USING(track_id);

 * sqlite:///chinook.db
Done.


purchased_tracks,not_purchased_tracks,total_tracks
1806,1697,3503


Out of a total of **3503 available tracks** only **1806 tracks were purchased at least once** while other **1697 were never purchased**.